In [ ]:
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from data import DIV2K
from model.edsr import edsr
from train import EdsrTrainer
%matplotlib inline
tf.enable_eager_execution()


In [ ]:
depth = 16

scale = 4
downgrade = 'bicubic'

In [ ]:
weights_dir = f'weights/edsr-{depth}-x{scale}'
weights_file = os.path.join(weights_dir, 'weights.h5')

os.makedirs(weights_dir, exist_ok=True)

In [ ]:
model = edsr(scale=scale, num_res_blocks=depth)
model.load_weights(weights_file)

In [ ]:
from model import resolve_single
from utils import load_image, plot_sample

def resolve_and_plot(x):
    lr = x
    sr = resolve_single(model, lr)
    return sr

In [ ]:
#function for resizing the image
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    dim = (width,height)
    resized = cv2.resize(image, dim, interpolation = inter)
    return resized

In [ ]:

#get the detected croped photos
def get_faces(filename, result_list):
    # load the image
    x = []
    data = pyplot.imread(filename)
    for i in range(len(result_list)):
        # get coordinates
        x1, y1, width, height = result_list[i]['box']
        x2, y2 = x1 + width, y1 + height
        x.append(data[y1-15:y2+15, x1-15:x2+15])
    return x



In [ ]:
import cv2
src = "/home/user/foider which contain images with multiple path/"#source folder for input images
dest = "/home/user/destination folder/"#destination folder for output images
count=0
fin_width=216
fin_height = 216
x = []
for path, subdirs, files in os.walk(src):
        for name in files:
            filename = os.path.join(path, name)
            if filename.endswith('.jpeg') or filename.endswith('.jpg') or filename.endswith(".png"):
                #print(filename)
                print("reading file : ",filename)
                pixels = pyplot.imread(filename)
                print("detecting faces in the image : "filename)
                detector = MTCNN()
                faces = detector.detect_faces(pixels)
                z = get_faces(filename, faces)
                x = np.array(z)
                for i in range(len(x)):
                    if not (( x[i].shape[0]==0) or (x[i].shape[1]==0)) and x[i].shape[2]==3:
                        result = x[i]
                        #if resolution is to two low use srgan for increasing resolution
                        if  x[i].shape[0]<fin_width+1 or x[i].shape[1]<fin_height+1 :
                            m =resolve_and_plot(x[i])
                            result = m.numpy()
                        resized_blue=image_resize(result,fin_width,fin_height)
                        resized_final = cv2.cvtColor(resized_blue, cv2.COLOR_BGR2RGB)
                        cv2.imwrite(dest+"newimage"+str(count)+".jpg", resized_final)
                        print("new image created from old image : ",filename)
                        count= count+1
